<a href="https://colab.research.google.com/github/uogbonda/US_Stores/blob/main/notebooks/colab-github-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Google Colab with GitHub



In [4]:
import warnings
from time import time
import numpy as np
from scipy import linalg
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.sparse import csr_matrix, issparse
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator
from sklearn.utils import check_random_state
from sklearn.utils._openmp_helpers import _openmp_effective_n_threads
from sklearn.utils.validation import check_non_negative
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
from sklearn import utils
# from sklearn import barnes_hut_tsne

In [8]:

MACHINE_EPSILON = np.finfo(np.double).eps

In [11]:
def _joint_probabilities(distances, desired_perplexity, verbose):
   distances = distances.astype(np.float32, copy=False)
   conditional_P = utils._binary_search_perplexity(distances, desired_perplexity, verbose)
   P = conditional_P + conditional_P.T
   sum_P = np.maximum(np.sum(P), MACHINE_EPSILON)
   P = np.maximum(squareform(P) / sum_P, MACHINE_EPSILON)
   return P

In [19]:
def _joint_probabilities_nn(distances, desired_perplexity, verbose):
  t0 = time()                         # Compute conditional probabilities such that they approximately match the desired perplexity.
  distances.sort_indices()
  n_samples = distances.shape[0]
  distances_data = distances.data.reshape(n_samples, -1)
  distances_data = distances_data.astype(np.float32, copy=False)
  conditional_P = utils._binary_search_perplexity(distances_data, desired_perplexity, verbose)
  assert np.all(np.isfinite(conditional_P)), "All probabilities should be finite"  # Symmetrize the joint probability distribution using sparse operations
  P = csr_matrix((conditional_P.ravel(), distances.indices, distances.indptr), shape=(n_samples, n_samples))
  P = P + P.T                               # Normalize the joint probability distribution
  sum_P = np.maximum(P.sum(), MACHINE_EPSILON)
  P /= sum_P
  assert np.all(np.abs(P.data) <= 1.0)
  if verbose >= 2:
    duration = time() - t0
    print("[t-SNE] Computed conditional probabilities in {:.3f}s".format(duration))
    return P


In [16]:

#!pip install core

#from core import    kl_cost_var, reverse_kl_cost_var, js_cost_var, hellinger_cost_var, chi_square_cost_var, p_Yp_Y_var_np, floath, find_sigma

In [17]:
def _kl_divergence(params,P,degrees_of_freedom,n_samples,n_components,skip_num_points=0,compute_error=True):
  X_embedded = params.reshape(n_samples, n_components)           # Q is a heavy-tailed distribution: Student's t-distribution
  dist = pdist(X_embedded, "sqeuclidean")
  dist /= degrees_of_freedom
  dist += 1.0
  dist **= (degrees_of_freedom + 1.0) / -2.0
  Q = np.maximum(dist / (2.0 * np.sum(dist)), MACHINE_EPSILON)         # Optimization trick below: np.dot(x, y) is faster than np.sum(x * y) because it calls BLAS.
  if compute_error:
    kl_divergence = 2.0 * np.dot(P, np.log(np.maximum(P, MACHINE_EPSILON) / Q))    # Objective: C (Kullback-Leibler divergence of P and Q)
  else:
    kl_divergence = np.nan                       # Gradient: dC/dY.  # pdist always returns double precision distances. Thus we need to take
    grad = np.ndarray((n_samples, n_components), dtype=params.dtype)
    PQd = squareform((P - Q) * dist)
    for i in range(skip_num_points, n_samples):
      grad[i] = np.dot(np.ravel(PQd[i], order="K"), X_embedded[i] - X_embedded)
      grad = grad.ravel()
      c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom
      grad *= c
      return kl_divergence, grad

In [18]:
def _kl_divergence_bh(params, P, degrees_of_freedom, n_samples, n_components, angle=0.5, skip_num_points=0, verbose=False, compute_error=True, num_threads=1):
   params = params.astype(np.float32, copy=False)
   X_embedded = params.reshape(n_samples, n_components)
   val_P = P.data.astype(np.float32, copy=False)
   neighbors = P.indices.astype(np.int64, copy=False)
   indptr = P.indptr.astype(np.int64, copy=False)
   grad = np.zeros(X_embedded.shape, dtype=np.float32)
   error = _barnes_hut_tsne.gradient(val_P, X_embedded, neighbors, indptr, grad, angle, n_components, verbose, dof=degrees_of_freedom, compute_error=compute_error, num_threads=num_threads)
   c = 2.0 * (degrees_of_freedom + 1.0) / degrees_of_freedom
   grad = grad.ravel()
   grad *= c
   return error, grad


In [20]:
def _gradient_descent(objective, p0, it, n_iter, n_iter_check=1, n_iter_without_progress=300, momentum=0.8, learning_rate=200.0, min_gain=0.01, min_grad_norm=1e-7, verbose=0, args=None, kwargs=None):
  if args is None:
    args = []
    if kwargs is None: 
      kwargs = {}
      p = p0.copy().ravel()
      update = np.zeros_like(p)
      gains = np.ones_like(p)
      error = np.finfo(float).max
      best_error = np.finfo(float).max
      best_iter = i = it
      tic = time()
      for i in range(it, n_iter):
        check_convergence = (i + 1) % n_iter_check == 0         # only compute the error when needed
        kwargs["compute_error"] = check_convergence or i == n_iter - 1
        error, grad = objective(p, *args, **kwargs)
        grad_norm = linalg.norm(grad)
        inc = update * grad < 0.0
        dec = np.invert(inc)
        gains[inc] += 0.2
        gains[dec] *= 0.8
        np.clip(gains, min_gain, np.inf, out=gains)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update
        if check_convergence:
          toc = time()
          duration = toc - tic
          tic = toc
          if verbose >= 2:
            print("[t-SNE] Iteration %d: error = %.7f," " gradient norm = %.7f" " (%s iterations in %0.3fs)" % (i + 1, error, grad_norm, n_iter_check, duration))
            if error < best_error:
              best_error = error
              best_iter = i
            elif i - best_iter > n_iter_without_progress:
              if verbose >= 2:
                print("[t-SNE] Iteration %d: did not make any progress " "during the last %d episodes. Finished." % (i + 1, n_iter_without_progress))
                break
            if grad_norm <= min_grad_norm:
              if verbose >= 2:
                print ("[t-SNE] Iteration %d: gradient norm %f. Finished." % (i + 1, grad_norm))
                break

    return p, error, i





In [21]:
def trustworthiness(X, X_embedded, *, n_neighbors=5, metric="euclidean"):
  dist_X = pairwise_distances(X, metric=metric)
  if metric == "precomputed":
    dist_X = dist_X.copy()         # we set the diagonal to np.inf to exclude the points themselves from their own neighborhood
    np.fill_diagonal(dist_X, np.inf)
    ind_X = np.argsort(dist_X, axis=1)    # "ind_X[i]"" is the index of sorted distances between i and other samples
    ind_X_embedded = (NearestNeighbors(n_neighbors=n_neighbors).fit(X_embedded).kneighbors(return_distance=False))
    n_samples = X.shape[0]
    inverted_index = np.zeros((n_samples, n_samples), dtype=int)
    ordered_indices = np.arange(n_samples + 1)
    inverted_index[ordered_indices[:-1, np.newaxis], ind_X] = ordered_indices[1:]
    ranks = (inverted_index[ordered_indices[:-1, np.newaxis], ind_X_embedded] - n_neighbors)
    t = np.sum(ranks[ranks > 0])
    t = 1.0 - t * (2.0 / (n_samples * n_neighbors * (2.0 * n_samples - 3.0 * n_neighbors - 1.0)))
    return t

In [22]:
class TSNE(BaseEstimator):
  EXPLORATION_N_ITER = 250
  N_ITER_CHECK = 50

  def __init__(self, n_components=2, *, perplexity=30.0, early_exaggeration=12.0, learning_rate="warn", n_iter=1000, n_iter_without_progress=300, min_grad_norm=1e-7, metric="euclidean", init="warn", verbose=0, random_state=None, method="barnes_hut", angle=0.5, n_jobs=None, square_distances="legacy"):
    self.n_components = n_components
    self.perplexity = perplexity
    self.early_exaggeration = early_exaggeration
    self.learning_rate = learning_rate
    self.n_iter = n_iter
    self.n_iter_without_progress = n_iter_without_progress
    self.min_grad_norm = min_grad_norm
    self.metric = metric
    self.init = init
    self.verbose = verbose
    self.random_state = random_state
    self.method = method
    self.angle = angle
    self.n_jobs = n_jobs              # TODO Revisit deprecation of square_distances for 1.1-1.3 (#12401)
    self.square_distances = square_distances



In [23]:
def _fit(self, X, skip_num_points=0):
  if isinstance(self.init, str) and self.init == "warn":
    warnings.warn("The default initialization in TSNE will change from 'random' to 'pca' in 1.2.", FutureWarning)
    self._init = "random"
  else:
    self._init = self.init
    if self.learning_rate == "warn":
      warnings.warn("The default initialization in TSNE will change from 200.0 to 'auto' in 1.2.", FutureWarning)
      self._learning_rate = 200.0
    else:
      self._learning_rate = self.learning_rate
      if isinstance(self._init, str) and self._init == "pca" and issparse(X):
        raise TypeError("PCA initialization is currently not supported with the sparse input matrix. Use 'init=random instead.'")
        if self.method not in ["barnes_hut", "exact"]:
          raise ValueError("'method' must be 'barnes_hut' or 'exact'")
          if self.angle < 0.0 or self.angle > 1.0:
            raise ValueError("'angle' must be between 0.0 - 1.0")
            if self.square_distances not in [True, "legacy"]:
              raise ValueError("'square_distances' must be True or 'legacy'.")
              if self._learning_rate == "auto":
                self._learning_rate = X.shape[0] / self.early_exaggeration / 4
                self._learning_rate = np.maximum(self._learning_rate, 50)
              else:
                if not (self._learning_rate > 0):
                  raise ValueError("'learning_rate' must be a positive number or 'auto'.")
                  if self.metric != "euclidean" and self.square_distances is not True:
                    warnings.warn("'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default settingwarning.", FutureWarning)
                    # removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by "
                    # default. Set 'square_distances'=True to silence this "
                    if self.method == "barnes_hut":
                      X = self._validate_data(X, accept_sparse=["csr"], ensure_min_samples=2, dtype=[np.float32, np.float64])
                    else:
                      X = self._validate_data(X, accept_sparse=["csr", "csc", "coo"], dtype=[np.float32, np.float64])
                      if self.metric == "precomputed":
                        if isinstance(self._init, str) and self._init == "pca":
                          raise ValueError('The parameter init="pca" cannot be used with metric="precomputed".')
                          if X.shape[0] != X.shape[1]:
                            raise ValueError("X should be a square distance matrix")
                            check_non_negative(X, "TSNE.fit(). With metric='precomputed', X should contain positive distances.")
                            if self.method == "exact" and issparse(X):
                              raise TypeError('TSNE with method="exact" does not accept sparse precomputed distance matrix. Use method="barnes_hut" or provide the dense distance matrix.')
                              if self.method == "barnes_hut" and self.n_components > 3:
                                raise ValueError("'n_components' should be inferior to 4 for the barnes_hut algorithm as it relies on quad-tree or oct-tree.")
                                random_state = check_random_state(self.random_state)
                                n_samples = X.shape[0]
                                neighbors_nn = None
                                if self.method == "exact":
                                  if self.metric == "precomputed":
                                    distances = X
                                  else:
                                    if self.verbose:
                                      print("[t-SNE] Computing pairwise distances...")  # Euclidean is squared here, rather than using **= 2,because euclidean_distances already calculates
                                      if self.metric == "euclidean":
                                         distances = pairwise_distances(X, metric=self.metric, squared=True) # squared distances, and returns np.sqrt(dist) for # squared=False. Also, Euclidean is slower for n_jobs>1, so don't set here
                                      else:
                                        distances = pairwise_distances(X, metric=self.metric, n_jobs=self.n_jobs)
                                        if np.any(distances < 0):
                                          raise ValueError("All distances should be positive, the metric given is not correct")
                                          if self.metric != "euclidean" and self.square_distances is True:
                                            distances **= 2
                                            P = _joint_probabilities(distances, self.perplexity, self.verbose)
                                            assert np.all(np.isfinite(P)), "All probabilities should be finite"
                                            assert np.all(P >= 0), "All probabilities should be non-negative"
                                            assert np.all(P <= 1), "All probabilities should be less or then equal to one"
                                          else:
                                             n_neighbors = min(n_samples - 1, int(3.0 * self.perplexity + 1))
                                             if self.verbose:
                                               print("[t-SNE] Computing {} nearest neighbors...".format(n_neighbors))
                                               # Find the nearest neighbors for every point
                                               knn = NearestNeighbors( algorithm="auto", n_jobs=self.n_jobs, n_neighbors=n_neighbors,  metric=self.metric)
                                               t0 = time()
                                               knn.fit(X)
                                               duration = time() - t0
                                               if self.verbose:
                                                 print("[t-SNE] Indexed {} samples in {:.3f}s...".format(n_samples, duration))
                                                 t0 = time()
                                                 distances_nn = knn.kneighbors_graph(mode="distance")
                                                 duration = time() - t0
                                                 if self.verbose:
                                                   print("[t-SNE] Computed neighbors for {} samples in {:.3f}s...".format(n_samples, duration))
                                                   del knn
                                                   if self.square_distances is True or self.metric == "euclidean":
                                                      distances_nn.data **= 2
                                                      P = _joint_probabilities_nn(distances_nn, self.perplexity, self.verbose)
                                                      if isinstance(self._init, np.ndarray):
                                                        X_embedded = self._init
                                                      elif self._init == "pca":
                                                        pca = PCA(n_components=self.n_components, svd_solver="randomized", random_state=random_state)
                                                        X_embedded = pca.fit_transform(X).astype(np.float32, copy=False)
                                                      elif self._init == "random":
                                                         X_embedded = 1e-4 * random_state.randn(n_samples, self.n_components).astype(np.float32)
                                                      else:
                                                        raise ValueError("'init' must be 'pca', 'random', or a numpy array")
                                                        degrees_of_freedom = max(self.n_components - 1, 1)
                                                        return self._tsne(P, degrees_of_freedom, n_samples, X_embedded=X_embedded, neighbors=neighbors_nn, skip_num_points=skip_num_points)
            

In [24]:
def _tsne(self, P, degrees_of_freedom, n_samples, X_embedded, neighbors=None, skip_num_points=0):
  opt_args = {"it": 0, "n_iter_check": self._N_ITER_CHECK, "min_grad_norm": self.min_grad_norm, "learning_rate": self._learning_rate, "verbose": self.verbose, "kwargs": dict(skip_num_points=skip_num_points), "args": [P, degrees_of_freedom, n_samples, self.n_components], "n_iter_without_progress": self._EXPLORATION_N_ITER, "n_iter": self._EXPLORATION_N_ITER, "momentum": 0.5}
  if self.method == "barnes_hut":
    obj_func = _kl_divergence_bh
    opt_args["kwargs"]["angle"] = self.angle
    # Repeat verbose argument for _kl_divergence_bh
    opt_args["kwargs"]["verbose"] = self.verbose
    # Get the number of threads for gradient computation here to
    # avoid recomputing it at each iteration.
    opt_args["kwargs"]["num_threads"] = _openmp_effective_n_threads()
  else:
    obj_func = _kl_divergence
    # Learning schedule (part 1): do 250 iteration with lower momentum but
    # higher learning rate controlled via the early exaggeration parameter
    P *= self.early_exaggeration
    params, kl_divergence, it = _gradient_descent(obj_func, params, **opt_args)
    if self.verbose:
      print("[t-SNE] KL divergence after %d iterations with early exaggeration: %f" % (it + 1, kl_divergence))
      # Learning schedule (part 2): disable early exaggeration and finish
      # optimization with a higher momentum at 0.8
      P /= self.early_exaggeration
      remaining = self.n_iter - self._EXPLORATION_N_ITER
      if it < self._EXPLORATION_N_ITER or remaining > 0:
        opt_args["n_iter"] = self.n_iter
        opt_args["it"] = it + 1
        opt_args["momentum"] = 0.8
        opt_args["n_iter_without_progress"] = self.n_iter_without_progress
        params, kl_divergence, it = _gradient_descent(obj_func, params, **opt_args)
        # Save the final number of iterations
        self.n_iter_ = it
        if self.verbose:
            print("[t-SNE] KL divergence after %d iterations: %f" % (it + 1, kl_divergence))
            X_embedded = params.reshape(n_samples, self.n_components)
            self.kl_divergence_ = kl_divergence
            return X_embedded
            


In [25]:
def fit_transform(self, X, y=None):
  embedding = self._fit(X)
  self.embedding_ = embedding
  return self.embedding_

In [26]:
def fit(self, X, y=None):
  self.fit_transform(X)
  return self


[Google Colaboratory](http://colab.research.google.com) is designed to integrate cleanly with GitHub, allowing both loading notebooks from github and saving notebooks to github.

## Loading Public Notebooks Directly from GitHub

Colab can load public github notebooks directly, with no required authorization step.

For example, consider the notebook at this address: https://github.com/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb.

The direct colab link to this notebook is: https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb.

To generate such links in one click, you can use the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) Chrome extension.

## Browsing GitHub Repositories from Colab

Colab also supports special URLs that link directly to a GitHub browser for any user/organization, repository, or branch. For example:

- http://colab.research.google.com/github will give you a general github browser, where you can search for any github organization or username.
- http://colab.research.google.com/github/googlecolab/ will open the repository browser for the ``googlecolab`` organization. Replace ``googlecolab`` with any other github org or user to see their repositories.
- http://colab.research.google.com/github/googlecolab/colabtools/ will let you browse the main branch of the ``colabtools`` repository within the ``googlecolab`` organization. Substitute any user/org and repository to see its contents.
- http://colab.research.google.com/github/googlecolab/colabtools/blob/master will let you browse ``master`` branch of the ``colabtools`` repository within the ``googlecolab`` organization. (don't forget the ``blob`` here!) You can specify any valid branch for any valid repository.

## Loading Private Notebooks

Loading a notebook from a private GitHub repository is possible, but requires an additional step to allow Colab to access your files.
Do the following:

1. Navigate to http://colab.research.google.com/github.
2. Click the "Include Private Repos" checkbox.
3. In the popup window, sign-in to your Github account and authorize Colab to read the private files.
4. Your private repositories and notebooks will now be available via the github navigation pane.

## Saving Notebooks To GitHub or Drive

Any time you open a GitHub hosted notebook in Colab, it opens a new editable view of the notebook. You can run and modify the notebook without worrying about overwriting the source.

If you would like to save your changes from within Colab, you can use the File menu to save the modified notebook either to Google Drive or back to GitHub. Choose **File→Save a copy in Drive** or **File→Save a copy to GitHub** and follow the resulting prompts. To save a Colab notebook to GitHub requires giving Colab permission to push the commit to your repository.

## Open In Colab Badge

Anybody can open a copy of any github-hosted notebook within Colab. To make it easier to give people access to live views of GitHub-hosted notebooks,
colab provides a [shields.io](http://shields.io/)-style badge, which appears as follows:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

The markdown for the above badge is the following:

```markdown
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
```

The HTML equivalent is:

```HTML
<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
```

Remember to replace the notebook URL in this template with the notebook you want to link to.